# Bronze Layer: DBFS CSV → P2T2.bronze Delta 적재

> **Purpose:** DBFS에 업로드된 CSV/이미지를 그대로 Bronze Layer Delta 테이블로 적재합니다.
> - 환자 바이탈 CSV (`vital_signs.csv`)
> - DICOM 메타데이터 CSV (`dicom_metadata.csv`)
> - 응급 데이터 CSV (`emergency_data.csv`)
> - 진료 기록 CSV (`medical_history.csv`)
>
> **카탈로그:** `P2T2` (Unity Catalog)
> **스키마:** `bronze`
> **저장 위치:** ADLS `2dt015-personal`

## 0. 카탈로그 설정

In [ ]:
spark.sql("USE CATALOG P2T2")
print("✅ Catalog: P2T2")

## 1. 라이브러리 및 설정

In [ ]:

from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime

# DBFS 경로 (SDK로 업로드된 위치)
DBFS_BASE = "dbfs:/patient_data"

# Bronze 테이블 매핑
TABLES = {
    "vital_signs":    f"{DBFS_BASE}/vital_signs.csv",
    "dicom_metadata":  f"{DBFS_BASE}/dicom_metadata.csv",
    "emergency_data":  f"{DBFS_BASE}/emergency_data.csv",
    "medical_history": f"{DBFS_BASE}/medical_history.csv",
}

print(f"🔧 Bronze Ingestion 설정: {len(TABLES)}개 테이블")
for name, path in TABLES.items():
    print(f"   {name} ← {path}")

## 2. CSV → Bronze Delta 적재

In [ ]:

def ingest_csv_to_bronze(table_name, csv_path):
    """
    DBFS의 CSV 파일을 P2T2.bronze Delta 테이블로 적재합니다.
    
    Args:
        table_name: 테이블명 (예: vital_signs)
        csv_path: DBFS CSV 경로
    """
    df = (spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(csv_path)
    )
    
    # 적재 메타데이터 추가
    df = df.withColumn("processed_at", F.current_timestamp())
    
    # Delta 테이블 저장
    full_table = f"P2T2.bronze.{table_name}"
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(full_table)
    
    count = spark.table(full_table).count()
    print(f"✅ {full_table}: {count:,} rows")
    return count

# 실행
total = 0
for name, path in TABLES.items():
    total += ingest_csv_to_bronze(name, path)

print(f"\n📦 Bronze 적재 완료: 총 {total:,} rows, {len(TABLES)}개 테이블")

## 3. 적재 상태 요약

In [ ]:

print("=" * 60)
print("📦 P2T2.bronze 적재 현황")
print("=" * 60)

tables = spark.sql("SHOW TABLES IN bronze").collect()
for t in tables:
    name = t['tableName']
    cnt = spark.table(f"bronze.{name}").count()
    cols = spark.table(f"bronze.{name}").columns
    print(f"  ✅ bronze.{name}: {cnt:,} rows, {len(cols)} columns")
    print(f"     Columns: {cols}")

print("=" * 60)